# Download the data and prepare it for analysis

## Project description
We decided to open a small robot-run cafe in Los Angeles. The project is promising but expensive, so  decide to try to attract investors. They’re interested in the current market conditions — will we be able to maintain your success when the novelty of robot waiters wears off?
We been asked to prepare some market research. We have open-source data on restaurants in LA.

## Import

In [ ]:
!pip install usaddress

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import datetime as dt
import usaddress

%matplotlib inline

## Load data

In [ ]:
try:
    rest_data = pd.read_csv('rest_data_us.csv', sep=',')
except:
    rest_data = pd.read_csv('/datasets/rest_data_us.csv', sep=',')

## Prepare data for analysis

- object_name — establishment name
- chain — chain establishment (TRUE/FALSE)
- object_type — establishment type
- address — address
- number — number of seats

In [ ]:
rest_data.head()

In [ ]:
rest_data.info()

Check the missing values rows

In [ ]:
missing_rows = rest_data[rest_data.isna().any(axis=1)]
missing_rows

Check if these establishment name appear elsewhere in the data. If so we will apply the chain type of them. else, we will delete it.

In [ ]:
array = missing_rows['object_name']
df = rest_data.loc[rest_data['object_name'].isin(array)]
df

They appear only once. Since we don't have important information about them we will delete them

In [ ]:
rest_data.dropna(inplace=True)

In [ ]:
rest_data[rest_data.isna().any(axis=1)]

ok

Change chain to categorical instead of object

In [ ]:
rest_data['chain'] = rest_data['chain'].astype('category')

Check if id numbers are unique

In [ ]:
rest_data['id'].nunique()

Yes, id numbers are unique

Lets see how the chain diverse

In [ ]:
rest_data['chain'].value_counts()

Looks ok

Lets see if we have duplicates of same establishment in the same address

In [ ]:
rest_data.groupby(['object_name', 'address'])['id'].count()

There are no more than one establishment in the same address. 

# Step 2. Data analysis

## Investigate the proportions of the various types of establishments. Plot a graph.


We'll plot a pie plot to see proportion

In [ ]:
df = rest_data.groupby('object_type')['id'].count().reset_index()
fig = px.pie(df, names='object_type', values='id', title='Proportions of various types of establishments')
fig.show()

Most popular establishments is restaurant

## Investigate the proportions of chain and nonchain establishments. Plot a graph.


We'll plot a pie plot to see proportion

In [ ]:
df = rest_data.groupby('chain')['id'].count().reset_index()
fig = px.pie(df, names='chain', values='id', title='Proportions of chain and nonchain establishments')
fig.show()

Larger amount of the establishments are non chain

## Which type of establishment is typically a chain?


In [ ]:
df = rest_data.pivot_table(
    index=['object_type', 'chain'],
    values='id',
    aggfunc='count'
).reset_index()
df

Bakery is **always** a chain

In [ ]:
rest_data['count'] = 1
fig = px.sunburst(rest_data, path=['object_type', 'chain'], values='count')
fig.show()

We can see the share of chains (true) and non chain (false) from all type of places. We can see bakery are only true since they all chains.

## What characterizes chains: many establishments with a small number of seats or a few establishments with a lot of seats?

In [ ]:
df = rest_data.loc[rest_data['chain']==True].groupby(
    'object_name').agg({'id':'count', 'number':'mean'}).reset_index()
df.columns = ['object_name', 'number_of_restaurants', 'average_number_of_seats']
fig = px.scatter(df, x="number_of_restaurants", y="average_number_of_seats")
fig.update_layout(
    title="establishments characterize by number of restaurants and seats"
)
fig.show()



Chains characterize by few establishments with a lot of seats rather than many establishments with a small number of seats

## Determine the average number of seats for each type of restaurant. On average, which type of restaurant has the greatest number of seats? Plot graphs.

In [ ]:
df = rest_data.groupby('object_type')['number'].mean().reset_index().sort_values(by='number')
fig = px.bar(df, x='object_type', y='number')
fig.update_layout(
    title="Number of seats for each type of restaurant",
    xaxis_title="type of restaurant",
    yaxis_title="average number of seats"   
)
fig.show()
df

Restaurant has the greatest number of seats

## Put the data on street names from the address column in a separate column.

Use a function to take just the street name 

In [ ]:
def cleaning_final(raw):
    if raw.startswith('OLVERA'):
        clean_adress='OLVERA,Los Angeles,USA'
    elif raw.startswith('1033 1/2 LOS ANGELES'):
        clean_adress='1033 1/2 LOS ANGELES ST,Los Angeles,USA'
    else:
        raw_address=usaddress.parse(raw)
        dict_address={}
        for i in raw_address:
            dict_address.update({i[1]:i[0]})
        clean_adress=str(dict_address['StreetName'])
    return clean_adress

In [ ]:
rest_data['clean_street_final']=rest_data.address.apply(cleaning_final)
rest_data.sample(10)

Worked!

## Plot a graph of the top ten streets by number of restaurants.

In [ ]:
df = rest_data.groupby(
    'clean_street_final')['id'].count().reset_index().sort_values(by='id', ascending=False).head(10)
fig = px.bar(df, x='clean_street_final', y='id')
fig.update_layout(
    title="Top ten streets by number of restaurants",
    xaxis_title="streets name",
    yaxis_title="number of restaurants"   
)
fig.show()

On the lead Sunset with 404 establishments and Wilshire with 398 establishments.

## Find the number of streets that only have one restaurant.

In [ ]:
number_of_streets = rest_data.groupby(
    'clean_street_final'
)['id'].count().reset_index().sort_values(by='id').query('id == 1').shape[0]
f'There are {number_of_streets} streets with only one restaurant.'

## For streets with a lot of restaurants, look at the distribution of the number of seats. What trends can you see?

In [ ]:
# Let's split the data to have only information about the 10 most busy streets
busy_streets = rest_data.groupby(
    'clean_street_final')['id'].count().reset_index().sort_values(by='id', ascending=False).head(10)
# get array of the busy streets
array = busy_streets['clean_street_final'].to_list()
# filter df by these streets 
df = rest_data.loc[rest_data['clean_street_final'].isin(array)]
fig = px.histogram(df, x="number")
fig.update_layout(
    title="Distribution of the number of seats")
fig.show()


Most of places have less than 50 seats

In [ ]:
df

In [ ]:
fig = px.box(df, x="number", y="object_type")
fig.show()

Restaurants and bars have the largest amount of seats. On bars as apposed to restaurants except of one location all the places are close by number of seats. 

In [ ]:
fig = px.box(df, x="number", y="clean_street_final")
fig.show()

#### On Wilshire and Hollywood there is the largest difference between the first quartile and the 4th (IQR) 

## Draw an overall conclusion and provide recommendations on restaurant type and number of seats. Comment on the possibility of developing a chain.

We don't have a measurement of success to the places we see on the data. We can only tell few characteristics like location, number of seats, part of a branch and type of restaurant. We will start from a premise that the  more common a particular type it is the more likely it is to be successful and otherwise there would have been few of it. So we will recommend to open a restaurant from type restaurant. Put between 45 to 50 seats. Set the place in one of the 10 most crowded streets. Since the majority of restaurant from type restaurant are not part of chain we will recommend not to develop in this direction.

# Step 3. Preparing a presentation

Presentation: <https://drive.google.com/file/d/1cEvIlgGtzADGJ2k-fljdDgFS8xeSv2yH/view?usp=sharing>